In [0]:
# In this example we read a footbal (soccer) game stat and we create a narration about the game as we are running a podcast
# use this website as an example: https://understat.com/match/27683

In [0]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display
from bs4 import BeautifulSoup
from openai import OpenAI

In [0]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print(
        "An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print(
        "An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [0]:
openai = OpenAI()
url = "https://understat.com/match/27683"

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

In [0]:
system_prompt = ("You are a football (soccer) analyst. Yuo are used to read stats of football \
                 games and extract relevant information. You are asked to be a podcast host and \
                  you need to create a narration of the game based on the stats you read and based \
                  on the play by play moves (the one with minutes upfront). You're talking to the \
                 general audience so try to use a easy language and do not be too much telegraphic")

In [0]:
# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


In [0]:
def create_user_prompt(game):
    user_prompt = f"You are looking at {game.title} football game"
    user_prompt += "\nThis is the entire webpage of the game \
    Please provide a narration of the game in  markdown. \
    Focus only on what happened on the game and the stats and ignore all the standings and anything else.\n\n"
    user_prompt += game.text
    return user_prompt


In [0]:
# Define messages with system_prompt and user_prompt
def messages_for(system_prompt_input, user_prompt_input):
    return [
        {"role": "system", "content": system_prompt_input},
        {"role": "user", "content": user_prompt_input}
    ]

In [0]:
# And now: call the OpenAI API.
game = Website(url)

response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages_for(system_prompt, create_user_prompt(game))
)

# Response is provided in Markdown and displayed accordingly
display(Markdown(response.choices[0].message.content))